<center><font size=6><b>GEOS 669 Geodetic Methods and Modeling</b></font></center>

# Lab 5: GPS Kinematic Position Estimates


**Assignment Due Date**: October 11th, 2023

Today's lab will be focused on determining kinematic positions from GPS data.

Your deliverable for this lab will be this notebook, with additional sections as outlined below. Please rename the notebook from `GEOS669-Lab5.ipynb` to `<last_name>_GEOS669-Lab5.ipynb`. For example, I (Emily Graves) would rename the notebook to `Graves_GEOS669-Lab5.ipynb`. You can do this by clicking on the filename in the top-left corner of this window (similar to, e.g., Google Docs). Download the completed file and image by checking its box in the JupyterHub file browser and selecting "Download" from the menu that appears or in the document's "File" menu under "Download as". Then submit it to Canvas under the Lab 5 assignment **no later than 1pm October 11th**.

### Introduction

Today we shift our focus from static positioning to kinematic, i.e. subdaily, processing. You will process 1 Hz data recorded at 2 GPS stations near the 2021 Mw 8.2 Chignik earthquake in Alaska.

### SET UP

You've installed and set up the processing environment last week and a few key files for this lab came from the git repository. You're all set if you worked through Lab 04!

Open a terminal session on the main Jupyter Hub page and `cd` into `GEOS669_geodesy/labs/Lab05.`

You're now in a directory for this lab. Stay there. You will work in the terminal window for this lab.


### Step 1: Data Retrieval

First, you'll need to get rinex files for the 2 stations we will process: AC13 and AC21. You can get the data from UNAVCO's high-rate archive. The data are organized by YEAR and DAY OF YEAR directories. First, get the day of year value for the day of the earthquake (search for it, if you don't know it, and **make sure to use UTC time**). We have a program that does just that conversion:

`date2doy YYYY-MM-DD`
	
The argument should be the 4-digit year, 2-digit month, 2-digit day of month. Separated by dashes, all in one string.

Armed with the correct day of year (doy) you can now ahead and download the data for the 2 stations (make sure to replace DOY with the value you've just calculated):

```
wget http://gps.alaska.edu/geos669/hr/DOY/ac13/ac13****.21d.Z
wget http://gps.alaska.edu/geos669/hr/DOY/ac21/ac21****.21d.Z

```


You should now have 2 `.Z` files in your directory, unzip them.

`gunzip *.Z`
    
You should now have 2 files in your directory (the stars indicate the DOY - I am not quite giving this away :) ...

<code style="background-color:#FFFFFF;font-size:smaller;">
    $ ls
    ac13***0.21d  ac21***0.21d  GEOS669-Lab5.ipynb  kinematic_tree  ocnload  run_kinematic.sh
</code>

### Step 2: Pre-Process

Since we'll be doing more advanced processing today, we'll need to include a few extra steps before we can go from the observables to position time series. First, we'll create one station database with the meta information from the rinex files:

`rinex2StaDb.py ac13***0.21d ac21***0.21d -outFile ChignikDb`
	
Next, we'll get the orbit products. We didn't do this last time, because we only processed one station per day. Since we're processing more than one now, we should download these only once:

```run_
mkdir -p orbits/2021

cd orbits/2021

wget -nH -np -nd -nv -r -R 'index.html*' -A "2021-07-29*gz" https://sideshow.jpl.nasa.gov/pub/JPL_GNSS_Products/Final/2021/

cd -
```

Note that we made this directory that includes a subdirectory for the year, because GipsyX expects it like that. Also note that we changed into the directory, downloaded and changed back out of the directory.

Listing the orbit directory contents should give you the list below. Important is that we have the `_hr` products, which are high-rate products needed for our processing.


`ls orbits/2021/`
<code style="background-color:#FFFFFF;font-size:smaller;">
2021-07-29.eo.gz         2021-07-29_nf.pos.gz   2021-07-29_nnr_hr.tdp.gz  2021-07-29_nnr.x.gz     2021-07-29.tdp.gz
2021-07-29_hr.tdp.gz     2021-07-29_nf.quat.gz  2021-07-29_nnr.pos.gz     2021-07-29.pcm.gz       2021-07-29.wlpb.gz
2021-07-29.meta.gz       2021-07-29_nf.tdp.gz   2021-07-29_nnr.quat.gz    2021-07-29.pos.gz       2021-07-29.x.gz
2021-07-29_nf.eo.gz      2021-07-29_nf.wlpb.gz  2021-07-29_nnr.tdp.gz     2021-07-29.quat.gz
2021-07-29_nf_hr.tdp.gz  2021-07-29_nnr.eo.gz   2021-07-29_nnr.wlpb.gz    2021-07-29.shadhist.gz
</code>

# Step 3: Process

We're now ready to run this. Note that in the command below I am referencing a configuration file that is setup for this processing (it came with the `git pull`). Anyway, it's similar to the tree file that we've seen last time, there are a lot of knobs to turn and I will not go into the details here. 

The `git pull` also deliverd you with `run_kinematic.sh` a script to run the analysis for this week. Please read through the comments so you understand what's going on. You absolutely should read the `gd2e.py` help to understand what each command line flag does! You will need to call it with the correct `doy` on the command line. Here are the main steps:

- copy an ocean load coefficient file into a generic file name that gipsy expects for each site
- convert the rinex file into a data-record file and run data editing (clock fixing, cycle slip fixing etc)
- call gd2e.py to run the actual analysis (for just a fraction of the data we have available to focus on the earthquake)
- convert the result file into east-north-vertical with respect to the first epoch solution

Note that this script can easily be expanded to include the rinex download you're doing above. The only thing you'd need to figure out is how to calculate ocean loading coefficients. [This website](http://holt.oso.chalmers.se/loading/) provides a service to produce these.

**The processing will take a while!** (minutes, not hours)  It'll be a good idea to use the processing time to set yourself up to prepare the work below so you're ready once the processing is done.

Once the processing is done, **plot the contents of the ac13.env and ac21.env files** (very similar to last week's assignment). Do you see the step due to the earthquake? Measure the offset in the horizontals. 

**Make a map showing the epicenter, the station locations, and the motions of the stations in the form of vectors originating at the station location and insert it in your notebook**. Similar to the plots [here](https://spotlight.unavco.org/how-gps-works/gps-and-tectonics/gps-and-earthquakes.html). I am not particular about how you plot the vectors as long as the values are correct and you give me a scale vector that shows how long a vector needs to be for 10 cm of motion. Make sure your plots are included in the notebook you're submitting!

In [1]:
#write your first plotting script here

In [2]:
#write your second plotting script here

Add your map image/s into your Lab05 directory and use the code below to display your image/s:
<img src="image.png" width="50%" />
